In [1]:
# ===============================================
# 01_integrate_london_ocm.py  (fixed)
# Integrate OpenChargeMap (London) + global usage + borough density
# Outputs: uk_stations_enriched.csv
# No sklearn; robust to missing/constant/empty data.
# ===============================================

import pandas as pd
import numpy as np
from pathlib import Path
import re

# --------- File paths ----------
ocm_path   = Path("london_ev_chargepoints_openchargemap.csv")
usage_path = Path("Global_EV_Charging_Behavior_2024.csv")
pop_path   = Path("housing-density-borough.csv")
out_csv    = Path("uk_stations_enriched.csv")

# --------- Helpers ----------
def read_csv_smart(p: Path) -> pd.DataFrame:
    for enc in ("utf-8", "utf-8-sig", "cp1252"):
        try:
            return pd.read_csv(p, encoding=enc)
        except Exception:
            pass
    return pd.read_csv(p)

def pick(df: pd.DataFrame, candidates, required=False):
    """Pick first matching column (case/contains-insensitive)."""
    cmap = {str(c).lower().strip(): c for c in df.columns}
    for want in candidates:
        key = want.lower().strip()
        if key in cmap:
            return cmap[key]
        for have_l, have in cmap.items():
            if key in have_l:
                return have
    if required:
        raise KeyError(f"Missing columns {candidates}. Available: {list(df.columns)}")
    return None

def to_num(s): return pd.to_numeric(s, errors="coerce")

def minmax(series: pd.Series) -> pd.Series:
    s = pd.to_numeric(series, errors="coerce").astype(float)
    if s.size == 0: return s
    m, M = s.min(skipna=True), s.max(skipna=True)
    if (pd.isna(m) and pd.isna(M)) or M == m:
        return pd.Series(0.0, index=s.index)
    return (s - m) / (M - m)

def minmax_df(df: pd.DataFrame) -> pd.DataFrame:
    if df.shape[0] == 0: return df.copy()
    out = pd.DataFrame(index=df.index)
    for c in df.columns: out[c] = minmax(df[c])
    return out

# --------- 1) Read ----------
ocm   = read_csv_smart(ocm_path)
usage = read_csv_smart(usage_path)
pop   = read_csv_smart(pop_path)

# FIX: use .str.strip() (vectorized), not .strip()
for df in (ocm, usage, pop):
    for c in df.select_dtypes(include="object"):
        df[c] = df[c].astype(str).str.strip()

# --------- 2) OCM columns ----------
COL_ID   = pick(ocm, ["OCM_ID","id"], required=True)
COL_OP   = pick(ocm, ["OperatorA","Operator","Network"])
COL_WEB  = pick(ocm, ["OperatorWebsite","Website"])
COL_USE  = pick(ocm, ["UsageType"])
COL_STAT = pick(ocm, ["Status"])
COL_ISOP = pick(ocm, ["IsOperational"])
COL_A1   = pick(ocm, ["AddressLine1"])
COL_A2   = pick(ocm, ["AddressLine2"])
COL_TOWN = pick(ocm, ["Town","City"])
COL_PROV = pick(ocm, ["StateOrProvince","County","Region"])
COL_PC   = pick(ocm, ["Postcode","Post code","Postal Code"])
COL_CTRY = pick(ocm, ["Country"])
COL_LAT  = pick(ocm, ["Latitude","Lat"], required=True)
COL_LON  = pick(ocm, ["Longitude","Lon","Long"], required=True)
COL_TITLE= pick(ocm, ["Title","Site Name"])
COL_NPTS = pick(ocm, ["NumberOfPoints","Number Of Points"])
COL_CONN = pick(ocm, ["ConnectorTypes"])
COL_MAXK = pick(ocm, ["MaxPowerKW","Max Power KW"])
COL_ALLK = pick(ocm, ["AllConnectorPowersKW","All Connector Powers KW"])
COL_CSTAT= pick(ocm, ["ConnectionStatuses"])
COL_DP   = pick(ocm, ["DataProvider"])
COL_UPD  = pick(ocm, ["DateLastStatusUpdate"])
COL_VER  = pick(ocm, ["DateLastVerified"])
COL_SUB  = pick(ocm, ["SubmissionStatus"])

# Numerics
ocm[COL_LAT] = to_num(ocm[COL_LAT])
ocm[COL_LON] = to_num(ocm[COL_LON])
if COL_MAXK: ocm[COL_MAXK] = to_num(ocm[COL_MAXK])
if COL_NPTS: ocm[COL_NPTS] = to_num(ocm[COL_NPTS])

# --------- 3) UK usage means ----------
COL_COUNTRY_U = pick(usage, ["Country"])
COL_UTIL_U    = pick(usage, ["Station Utilization Rate (%)","Utilization Rate (%)","Utilisation Rate (%)"])
COL_ENERGY_U  = pick(usage, ["Energy Delivered (kWh)","Energy (kWh)","kWh"])

if COL_COUNTRY_U:
    mask = usage[COL_COUNTRY_U].str.lower().isin(
        ["united kingdom","uk","england","scotland","wales","northern ireland"]
    )
    usage_uk = usage[mask]
else:
    usage_uk = pd.DataFrame(columns=usage.columns)

if usage_uk.empty: usage_uk = usage.copy()

uk_avg_util   = float(usage_uk[COL_UTIL_U].mean(skipna=True))   if COL_UTIL_U in usage_uk else 0.0
uk_avg_energy = float(usage_uk[COL_ENERGY_U].mean(skipna=True)) if COL_ENERGY_U in usage_uk else 0.0
uk_n_rows     = int(usage_uk.shape[0])
if np.isnan(uk_avg_util):   uk_avg_util = 0.0
if np.isnan(uk_avg_energy): uk_avg_energy = 0.0

# --------- 4) Borough density (latest year) ----------
COL_BORO_P = pick(pop, ["Name","Borough"], required=True)
COL_YEAR_P = pick(pop, ["Year"])
COL_DENS_P = pick(pop, ["Population_per_square_kilometre","Population per square kilometre","Density"], required=True)

pop[COL_DENS_P] = to_num(pop[COL_DENS_P].astype(str).str.replace(",", "", regex=False))
if COL_YEAR_P: pop[COL_YEAR_P] = to_num(pop[COL_YEAR_P])

if COL_YEAR_P:
    pop_latest = pop.sort_values(COL_YEAR_P).groupby(COL_BORO_P, as_index=False).tail(1)
else:
    pop_latest = pop.drop_duplicates(subset=[COL_BORO_P], keep="last")

pop_latest.rename(columns={COL_BORO_P:"borough_name", COL_DENS_P:"borough_density_km2"}, inplace=True)
pop_latest["borough_key"] = pop_latest["borough_name"].str.lower()

# --------- 5) Extract London borough from text ----------
LONDON_BOROUGHS = [
    "City of London","Barking and Dagenham","Barnet","Bexley","Brent","Bromley","Camden","Croydon",
    "Ealing","Enfield","Greenwich","Hackney","Hammersmith and Fulham","Haringey","Harrow","Havering",
    "Hillingdon","Hounslow","Islington","Kensington and Chelsea","Kingston upon Thames","Lambeth",
    "Lewisham","Merton","Newham","Redbridge","Richmond upon Thames","Southwark","Sutton",
    "Tower Hamlets","Waltham Forest","Wandsworth","Westminster"
]
ALIASES = {
    "city of westminster":"Westminster",
    "hammersmith & fulham":"Hammersmith and Fulham",
    "kensington & chelsea":"Kensington and Chelsea",
    "richmond upon thames":"Richmond upon Thames",
    "kingston upon thames":"Kingston upon Thames",
}

boro_patterns = [(b, re.compile(rf"\b{re.escape(b)}\b", re.I)) for b in LONDON_BOROUGHS]
alias_patterns = [(ALIASES[k], re.compile(rf"\b{k}\b", re.I)) for k in ALIASES]

def extract_borough(row) -> str:
    fields = []
    for c in [COL_A1, COL_A2, COL_TITLE, COL_TOWN, COL_PROV]:
        if c and c in row and pd.notna(row[c]):
            fields.append(str(row[c]))
    text = " | ".join(fields).lower()
    for b, pat in boro_patterns:
        if pat.search(text): return b
    for b, pat in alias_patterns:
        if pat.search(text): return b
    return "Unknown"

ocm["borough"] = ocm.apply(extract_borough, axis=1)

# --------- 6) Shape station table ----------
stations = pd.DataFrame({
    "ocm_id": ocm[COL_ID],
    "operator": ocm[COL_OP] if COL_OP else np.nan,
    "usage_type": ocm[COL_USE] if COL_USE else np.nan,
    "status": ocm[COL_STAT] if COL_STAT else np.nan,
    "is_operational": ocm[COL_ISOP] if COL_ISOP else np.nan,
    "address1": ocm[COL_A1] if COL_A1 else np.nan,
    "address2": ocm[COL_A2] if COL_A2 else np.nan,
    "town": ocm[COL_TOWN] if COL_TOWN else np.nan,
    "state_province": ocm[COL_PROV] if COL_PROV else np.nan,
    "postcode": ocm[COL_PC] if COL_PC else np.nan,
    "country": ocm[COL_CTRY] if COL_CTRY else np.nan,
    "latitude": ocm[COL_LAT],
    "longitude": ocm[COL_LON],
    "title": ocm[COL_TITLE] if COL_TITLE else np.nan,
    "num_points": ocm[COL_NPTS] if COL_NPTS else np.nan,
    "connector_types": ocm[COL_CONN] if COL_CONN else np.nan,
    "max_power_kw": ocm[COL_MAXK] if COL_MAXK else np.nan,
    "all_connector_powers_kw": ocm[COL_ALLK] if COL_ALLK else np.nan,
    "connection_statuses": ocm[COL_CSTAT] if COL_CSTAT else np.nan,
    "data_provider": ocm[COL_DP] if COL_DP else np.nan,
    "last_status_update": ocm[COL_UPD] if COL_UPD else np.nan,
    "last_verified": ocm[COL_VER] if COL_VER else np.nan,
    "submission_status": ocm[COL_SUB] if COL_SUB else np.nan,
    "borough": ocm["borough"]
})

# --------- 7) Merge borough density ----------
stations["borough_key"] = stations["borough"].astype(str).str.strip().str.lower()
stations = stations.merge(pop_latest[["borough_key","borough_density_km2"]],
                          on="borough_key", how="left")

density_mean = float(pop_latest["borough_density_km2"].mean(skipna=True))
stations["borough_density_km2"] = stations["borough_density_km2"].fillna(density_mean)

# --------- 8) Attach UK usage means ----------
stations["uk_avg_util_pct"]   = uk_avg_util
stations["uk_avg_energy_kWh"] = uk_avg_energy
stations["uk_usage_rows"]     = uk_n_rows

# --------- 9) Features & score ----------
feat = pd.DataFrame({
    "util": stations["uk_avg_util_pct"],
    "density": stations["borough_density_km2"],
    "power": stations["max_power_kw"],
    "points": stations["num_points"]
})
feat = feat.fillna(feat.median(numeric_only=True))
feat_scaled = minmax_df(feat).fillna(0.0)

stations["priority_score"] = (
    0.35 * feat_scaled["density"] +
    0.30 * feat_scaled["power"] +
    0.25 * feat_scaled["util"] +
    0.10 * feat_scaled["points"]
).fillna(0.0)

# --------- 10) Save ----------
keep_cols = [
    "ocm_id","operator","usage_type","status","is_operational",
    "address1","address2","town","state_province","postcode","country",
    "latitude","longitude","title","num_points","connector_types",
    "max_power_kw","all_connector_powers_kw","connection_statuses",
    "borough","borough_density_km2",
    "uk_avg_util_pct","uk_avg_energy_kWh","uk_usage_rows",
    "priority_score","last_status_update","last_verified",
    "submission_status","data_provider"
]
stations[keep_cols].to_csv(out_csv, index=False)

# --------- 11) Summary ----------
print("\n=== London OCM Integration (fixed) ===")
print(f"OCM rows in:       {len(ocm):,}")
print(f"Stations out:      {len(stations):,}")
print(f"Borough 'Unknown': {(stations['borough'] == 'Unknown').sum():,}")
print(f"UK mean util %:    {uk_avg_util:.2f}")
print(f"UK mean energy kWh:{uk_avg_energy:.2f}")
print(f"Density mean used: {density_mean:.2f} people/km²")
print("Saved:", out_csv)
print("\nPreview:")
print(stations[keep_cols].head(5))



=== London OCM Integration (fixed) ===
OCM rows in:       10,000
Stations out:      10,000
Borough 'Unknown': 9,019
UK mean util %:    50.22
UK mean energy kWh:36.88
Density mean used: 9318.95 people/km²
Saved: uk_stations_enriched.csv

Preview:
   ocm_id            operator                                  usage_type  \
0  253415              EV Dot                    Public - Pay At Location   
1    4396  (Unknown Operator)                                         nan   
2   52877       BP Pulse (UK)                Public - Membership Required   
3  146490               VIRTA  Private - For Staff, Visitors or Customers   
4    4399       BP Pulse (UK)                Public - Membership Required   

        status is_operational         address1             address2  \
0          nan            nan   Rainsford Road                  nan   
1  Operational           True    Spring Garden          Westminster   
2  Operational           True   Spring Gardens  City of Westminster   
3  Ope